# Simple Notebook for Visualizing Accelerometer Data  of 10 Steps

In [134]:
import pandas as pd
import numpy as np
from scipy.signal import butter,filtfilt,find_peaks
import plotly.graph_objects as go


## Read the Data

In [135]:
df_acc = pd.read_csv('../Data/Data_Nils/10_schritte/accelerometer.csv')
readings_np = df_acc.to_numpy()

## Implement a Butterworthfilter Helper Function

In [136]:
def butter_lowpass_filter(data):
    cutoff = 2
    order = 8
    normal_cutoff = cutoff / 100
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='lowpass', analog=False)
    y = filtfilt(b, a, data)
    return y

### Filter All Readings Besides the Time

In [137]:
filtered_readings_np = np.apply_along_axis(butter_lowpass_filter, 0, readings_np[:, 1:])

In [138]:
filtered_readings_np = np.c_[readings_np[:, 0], filtered_readings_np]

## Plot the Filtered Data

In [139]:
# Filter the data, and plot both the original and filtered signals.
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 1],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 2],
            line =  dict(shape =  'spline' ),
            name = 'y signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 3],
            line =  dict(shape =  'spline' ),
            name = 'z signal')
            )
fig.add_trace(go.Scatter(
            y = filtered_readings_np[:, 0],
            line =  dict(shape =  'spline' ),
            name = 'time signal')
            )
fig.show()

## Implement a Helper for the Peak Finder Function

In [140]:
def helper_find_peaks(data):
    ret, _ = find_peaks(data, height=0.1)
    return ret

## Find the peaks

In [141]:
# find peaks in absolute acceleration -> resulting in steps
peaks = []

for i in range(1, 4):
    peaks.append(helper_find_peaks(filtered_readings_np[:, i]))

peaks

[array([ 187, 1779, 1882, 2197, 2442, 2519, 2710, 2811, 3077, 3229, 3386],
       dtype=int64),
 array([1833, 2037, 2205, 2353, 2504, 2663, 2815, 2952, 3092, 3240, 3598],
       dtype=int64),
 array([1907, 2083, 2239, 2391, 2558, 2696, 2840, 2996, 3140, 3271],
       dtype=int64)]

## Show Peaks In Z-Axis

In [142]:
filtered_df = pd.DataFrame(filtered_readings_np, columns=['time', 'x', 'y', 'z', 'abs'])

In [143]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = filtered_df['z'],
            line =  dict(shape =  'spline' ),
            name = 'x signal')
            )
fig.add_trace(go.Scatter(
            x = peaks[2],
            y = filtered_df.loc[peaks[2]]['z'],
            mode =  'markers',
            name = 'peaks')
            )